In [3]:
import numpy as np
import sys
import matplotlib.pyplot

In [2]:
sys.version_info


sys.version_info(major=2, minor=7, micro=18, releaselevel='final', serial=0)

In [3]:
print "print zuado?"

print zuado?


In [2]:
from pacman import runGames,loadAgent
from pacman import Directions
from util import Queue
import textDisplay
import game
import layout
import random
import graphicsDisplay

qual das 5 direcoes ele toma

- absoluto 
- POV pacman

coisas uteis

- posicao pacman
- quais acoes sao validas
- a posiçao dos fantasma ( distancia aos fantasmas, distancia manhatan)
- a posiçao das comidas ( matriz pode ser grande)
- a posicao das capsulas
- os fantasmas tao assustados ou n

tomar decisao

- 1 equacao pra cada direcao
- emula que vc fez 1 acao legao
    - ve a pontucao = score - p1 * distancia (f1) - p2 * distancia (f2)
- pontua = score -(-1 se assustado) p1 * distancia (f1) - (-1 se assustado) p2 * distancia (f2)



In [5]:
class MyOwnAgent(game.Agent):
    "An agent that does whatever he feels like doing"
    
    def countScaredGhosts(self, state):
        total=0
        for ghost in state.getGhostStates():
            if(ghost.scaredTimer !=0):
                total+=1
        return total
    

    def getAction(self, state):
        # Generate candidate actions
        legal = state.getLegalPacmanActions()
        currentPos=state.getPacmanPosition()
        if Directions.STOP in legal: legal.remove(Directions.STOP)
    
        numCapsulesCurrentState=len(state.getCapsules())
        currentScaredGhosts=self.countScaredGhosts(state)
        successors = [(state.generateSuccessor(0, action), action) for action in legal]
        scored = [(self.scoreEvaluation(state,numCapsulesCurrentState,currentScaredGhosts,currentPos), action) for state, action in successors]
        bestScore = max(scored)[0]
        bestActions = [pair[1] for pair in scored if pair[0] == bestScore]
#         print(bestActions)
        
        return random.choice(bestActions)
    
    # distanca que calcula o numero de movimentos ate chegar no fantasma
    
    # score - so leva em conta a comida nos vizinhos do pacman
    # a posicao fantasma
    # a posicao das capsulas 
    
    # x aumenta pra direita
    # y aumenta pra cima
    
    # 0y       xy
    #
    # 00       x0
    
    def getLegalSteps(self, pos, walls, visited):
        coluna=pos[0]
        linha=pos[1]
        
        lista=[]
        if(coluna+1 <walls.width):
            if(walls[coluna+1][linha]==False and visited[coluna+1][linha]==0):
                lista.append((coluna+1,linha))
        if(coluna-1 >= 0):
            if(walls[coluna-1][linha]==False and visited[coluna-1][linha]==0):
                lista.append((coluna-1,linha))
        if(linha+1 <walls.height ):
            if(walls[coluna][linha+1]==False and visited[coluna][linha+1]==0):
                lista.append((coluna,linha+1))
        if(linha-1 >= 0):
            if(walls[coluna][linha-1]==False and visited[coluna][linha-1]==0):
                lista.append((coluna,linha-1))
        return lista
    
    def findNearestFood(self, state, posIni, posForbidden):
        walls= state.getWalls()
        foods= state.getFood()
        queue = Queue()
        
        
        visited=np.zeros((walls.width,walls.height))
        visited[posForbidden[0]][posForbidden[1]]=1
                
        matHeight=foods.height
        matWidth=foods.width
        
        queue.push((posIni, 1))

        while(not queue.isEmpty()):
            current=queue.pop()
            pos=current[0]
            stepsUntilNow=current[1]
            
            if(foods[pos[0]][pos[1]]==True):
                return stepsUntilNow
            
            visited[pos[0]][pos[1]]=1
            possibles=self.getLegalSteps(pos,walls, visited) # pos = (x,y), walls = matriz true/falso, visitados  [(2,2),(2,1),(2,3)]
            for possibleMove in possibles:
                queue.push( (possibleMove, stepsUntilNow+1))
        return -1
    
    def scoreEvaluation(self, state,numCapsulesCurrentState,currentScaredGhosts, currentPacmanPos):
        ghostPositions = state.getGhostPositions() # [(x,y),(x,y)]
        pacmanPosition = state.getPacmanPosition()# (x,y)
        capsulePositions = state.getCapsules() # [(x,y),(x,y)]
        score = state.getScore() # float
        dist1= abs(pacmanPosition[0] - ghostPositions[0][0]) + abs(pacmanPosition[1] - ghostPositions[0][1])
        dist2= abs(pacmanPosition[0] - ghostPositions[1][0]) + abs(pacmanPosition[1] - ghostPositions[1][1])
        
        fear1=(state.getGhostStates()[0].scaredTimer)
        fear2=(state.getGhostStates()[1].scaredTimer)
        
        scaredGhostsInThisState=self.countScaredGhosts(state)
        
        difScaredGhosts= currentScaredGhosts - scaredGhostsInThisState
        
        nearestFoodDistance= self.findNearestFood(state, pacmanPosition,currentPacmanPos)
        
        print("res",nearestFoodDistance)
        
        p1= 3
        p2= 3
        p3= 30
        p4= 1
        p5= 20
        p6= 100
        pfear= 1.5
        pEatGhost= 1000
        pNearestFood= 3
        pNearestFoodCeiling=30
        
        nearestFoodBonus= pNearestFood * (pNearestFoodCeiling-nearestFoodDistance)
        
        bonusEatGhost=0
        if(difScaredGhosts > 0):
            
            bonusEatGhost= pEatGhost * difScaredGhosts
        
        bonusIfFearGhost= (pfear * fear1 * (p3-dist1)) + (pfear * fear2 * (p3-dist2))
        
        
        bonusGivenByCapsuleComsumption = (numCapsulesCurrentState - len(capsulePositions)) * p6
        
        bonusByGhostDistance =(p1*(p3-dist1)) - (p2*(p3-dist2))
        
        totalCapsule=0
        for capsule in capsulePositions:
            distCapsule= abs(pacmanPosition[0] - capsule[0]) + abs(pacmanPosition[1] - capsule[1])
            totalCapsule+= p4*(p5-distCapsule)
        
        # 200 -  p1 * (p3 - 4)  - p2 * 7
        
        formula = score - bonusByGhostDistance + totalCapsule + bonusGivenByCapsuleComsumption + bonusIfFearGhost + bonusEatGhost + nearestFoodBonus
        return formula

In [6]:
props={}
pacman = MyOwnAgent()
ghostType = loadAgent('RandomGhost', True)
textDisplay.SLEEP_TIME = 0
props['layout'] = layout.getLayout( 'mediumClassic' )
props['pacman'] = pacman
props['ghosts'] = [ghostType( i+1 ) for i in range( 2 )]
# props['display'] = textDisplay.PacmanGraphics()
# props['display'] = textDisplay.NullGraphics()
props['display'] = graphicsDisplay.PacmanGraphics(1, frameTime = 0)
props['numGames'] = 5
props['record'] = False
props['catchExceptions'] = False
props['timeout'] = 30

In [7]:
#res=runGames(**props)

In [8]:
#runGames(**props)

In [9]:
#res[0].state.getScore()

In [10]:
#print(res[0].agentOutput[0].getvalue())

In [11]:
#dir(res[0].agentOutput[0])

In [11]:
class KeyboardAgentMeu(game.Agent):
    """
    An agent controlled by the keyboard.
    """
    # NOTE: Arrow keys also work.
    WEST_KEY  = 'a'
    EAST_KEY  = 'd'
    NORTH_KEY = 'w'
    SOUTH_KEY = 's'
    STOP_KEY = 'q'
    

    def __init__( self, index = 0 ):

        self.lastMove = Directions.STOP
        self.index = index
        self.keys = []
        
    def getLegalSteps(self, pos, walls, visited):
        coluna=pos[0]
        linha=pos[1]
        
        lista=[]
        if(coluna+1 <walls.width):
            if(walls[coluna+1][linha]==False and visited[coluna+1][linha]==0):
                lista.append((coluna+1,linha))
        if(coluna-1 >= 0):
            if(walls[coluna-1][linha]==False and visited[coluna-1][linha]==0):
                lista.append((coluna-1,linha))
        if(linha+1 <walls.height ):
            if(walls[coluna][linha+1]==False and visited[coluna][linha+1]==0):
                lista.append((coluna,linha+1))
        if(linha-1 >= 0):
            if(walls[coluna][linha-1]==False and visited[coluna][linha-1]==0):
                lista.append((coluna,linha-1))
        return lista
    
    def findNearestFood(self, state, posIni, posForbidden):
        walls= state.getWalls()
        foods= state.getFood()
        queue = Queue()
        
        
        visited=np.zeros((walls.width,walls.height))
        visited[posForbidden[0]][posForbidden[1]]=1
                
        matHeight=foods.height
        matWidth=foods.width
        
        queue.push((posIni, 1))

        while(not queue.isEmpty()):
            current=queue.pop()
            pos=current[0]
            stepsUntilNow=current[1]
            
            if(foods[pos[0]][pos[1]]==True):
                return stepsUntilNow
            
            visited[pos[0]][pos[1]]=1
            possibles=self.getLegalSteps(pos,walls, visited) # pos = (x,y), walls = matriz true/falso, visitados  [(2,2),(2,1),(2,3)]
            for possibleMove in possibles:
                queue.push( (possibleMove, stepsUntilNow+1))
        return -1
    def scoreEvaluation(self, state,numCapsulesCurrentState,currentScaredGhosts, currentPacmanPos):
        pacmanPosition = state.getPacmanPosition()# (x,y)      
        nearestFoodDistance= self.findNearestFood(state, pacmanPosition,currentPacmanPos)
        
        print("res",nearestFoodDistance)
        return 0
    
    def getAction( self, state):
        from graphicsUtils import keys_waiting
        from graphicsUtils import keys_pressed
        keys = keys_waiting() + keys_pressed()
        if keys != []:
            self.keys = keys

        legal = state.getLegalActions(self.index)
        move = self.getMove(legal)

        if move == Directions.STOP:
            # Try to move in the same direction as before
            if self.lastMove in legal:
                move = self.lastMove

        if (self.STOP_KEY in self.keys) and Directions.STOP in legal: move = Directions.STOP

        if move not in legal:
            move = random.choice(legal)

        self.lastMove = move
        
        currentPos = state.getPacmanPosition()# (x,y)
        
        successors = [(state.generateSuccessor(0, action), action) for action in legal]
        scored = [(self.scoreEvaluation(state,0,0,currentPos), action) for state, action in successors]

        return move

    def getMove(self, legal):
        move = Directions.STOP
        if   (self.WEST_KEY in self.keys or 'Left' in self.keys) and Directions.WEST in legal:  move = Directions.WEST
        if   (self.EAST_KEY in self.keys or 'Right' in self.keys) and Directions.EAST in legal: move = Directions.EAST
        if   (self.NORTH_KEY in self.keys or 'Up' in self.keys) and Directions.NORTH in legal:   move = Directions.NORTH
        if   (self.SOUTH_KEY in self.keys or 'Down' in self.keys) and Directions.SOUTH in legal: move = Directions.SOUTH
        return move

In [ ]:
props={}
#pacman = MyOwnAgent()
ghostType = loadAgent('RandomGhost', True)
textDisplay.SLEEP_TIME = 0

#pacmanType = loadAgent('KeyboardAgent', False)
#pacman = pacmanType()

pacman = KeyboardAgentMeu()

props['layout'] = layout.getLayout( 'mediumClassic' )
props['pacman'] = pacman
#props['ghosts'] = [ghostType( i+1 ) for i in range( 2 )]
props['ghosts'] = []

# props['display'] = textDisplay.PacmanGraphics()
# props['display'] = textDisplay.NullGraphics()
props['display'] = graphicsDisplay.PacmanGraphics(1, frameTime = 0.20)

props['numGames'] = 5
props['record'] = False
props['catchExceptions'] = False
props['timeout'] = 30

res=runGames(**props)

('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 2)
('res', 3)
('res', 2)
('res', 2)
('res', 4)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)
('res', 2)
('res', 2)
('res', 5)